## Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import datetime
import folium
sb.set()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data import

In [2]:
data = pd.read_csv("cleaned-Housing.csv")
data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,resale_year
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,1990
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,1990
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,1990
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,1990
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,1990


In [3]:
# Coordinate of each town
# Source: database.earth, latlong.info, latitude.to, www.gardencitysingapore.com
townCoords = {
    "ANG MO KIO": [1.3698, 103.8461],
    "BEDOK": [1.3236, 103.9273],
    "BISHAN": [1.3499986, 103.8499966],
    "BUKIT BATOK": [1.3496, 103.7528],
    "BUKIT MERAH": [1.2858300, 103.8194400],
    "BUKIT PANJANG": [1.3786, 103.7626],
    "BUKIT TIMAH": [1.3294, 103.8021],
    "CENTRAL AREA": [1.2895, 103.8143],
    "CHOA CHU KANG": [1.3833318, 103.749997],
    "CLEMENTI": [1.30918042994, 103.75914363],
    "GEYLANG": [1.3182, 103.8871],
    "HOUGANG": [1.3736, 103.8867],
    "JURONG EAST": [1.32399537068, 103.737298717],
    "JURONG WEST": [1.3396, 103.7073],
    "KALLANG/WHAMPOA": [1.3106, 103.866],
    "MARINE PARADE": [1.3030, 103.9072],
    "PASIR RIS": [1.3739, 103.9493],
    "PUNGGOL": [1.4051, 103.9023],
    "QUEENSTOWN": [1.2994400, 103.8058300],
    "SEMBAWANG": [1.4491, 103.8201],
    "SENGKANG": [1.3901, 103.8952],
    "SERANGOON": [1.3496, 103.8737],
    "TAMPINES": [1.3547, 103.9437],
    "TOA PAYOH": [1.3354, 103.8497],
    "WOODLANDS": [1.43801, 103.78877],
    "YISHUN": [1.4293, 103.8355]
}


## Map renderer

In [4]:
# Render map, can pass in filtered/specific data
# Radius based on count of each town
# To modify circle size, tweak `radiusScaler` parameter
def renderMap(data, radiusScaler=100, color="#0000AA"):
    global sgmap
    sgmap = folium.Map(location=[1.3666,103.800], zoom_start=12,tiles="OpenStreetMap",double_click_zoom=False)
    for townName in townCoords:
#         print(townName)
        count = 0
        for s in data["town"]:
            count += townName==s
        radius = count / radiusScaler + 0.000001
#         radius = data["town"].count(s)/10
        popup = townName + "\n" + str(count)
        folium.CircleMarker(location=townCoords[townName], radius=radius, color=color, fill=True, popup=popup).add_to(sgmap)

In [5]:
sgmap = folium.Map(location=[1.3666,103.800], zoom_start=12, tiles="OpenStreetMap")
# folium.CircleMarker(location=townCoords["SERANGOON"]).add_to(sgmap)
for s in townCoords:
    folium.CircleMarker(location=townCoords[s], radius=10, color="#0000AA", fill=True, popup=s).add_to(sgmap)
sgmap.save("regular-map.html")
sgmap

## Visualization

Let's take a look at how the resales is distributed accross Singapore. The map will be rendered together with orange circles.

In [6]:
renderMap(data, 1000, "#cc8400")
sgmap.save("all-map.html")
sgmap

We will split HDB based on the lease_commence_date into 2 categories, before 2000 and after. The maps will be rendered together with cyan circles.

In [7]:
renderMap(data[(1980 <= data['lease_commence_date']) & (data['lease_commence_date'] < 2000)], 1000, "#1c9c95")
sgmap.save("lease-before2000-map.html")
sgmap

In [8]:
renderMap(data[(2000 <= data['lease_commence_date']) & (data['lease_commence_date'] <= 2020)], 1000, "#1c9c95")
sgmap.save("lease-after2000-map.html")
sgmap

Comparing this two maps, we see that most resale happens for HDB with lease_commence_year before 2000. This may suggest that people tend to resale older HDB compared to newer ones. It's also possible that older HDB flats have certain advantages that make them more desirable for resale.

Another factor could be pricing. Older HDB flats might be priced lower than newer ones, making them more affordable for buyers, especially first-time homeowners or those on a tighter budget. This affordability factor could drive higher resale activity for older flats.

Now, we split the resale year into 3 groups, one for each decade. The maps will be rendered together with purple circles.

In [9]:
renderMap(data[(1990 <= data['resale_year']) & (data['resale_year'] < 2000)], 1000, "#8a2be2")
sgmap.save("resale-199X-map.html")
sgmap

In [10]:
renderMap(data[(2000 <= data['resale_year']) & (data['resale_year'] < 2010)], 1000, "#8a2be2")
sgmap.save("resale-200X-map.html")
sgmap

In [11]:
renderMap(data[(2010 <= data['resale_year']) & (data['resale_year'] <= 2020)], 1000, "#8a2be2")
sgmap.save("resale-201X-map.html")
sgmap

Here we can see that for most towns, they tend to decrease in terms of number of resale. There could be several reasons contributing to this trend.

One possible reason is changing demographics and preferences. Certain towns or areas may become less attractive to buyers over time due to shifts in population demographics or changes in lifestyle trends.

Infrastructure and development can also play a significant role. Towns that receive more investment in terms of facilities tend to attract more buyers, leading to higher resale activity. Conversely, towns that lack these amenities may experience a decline in demand for resale flats.

Next, we only present HDB resales that lies on the lower and higher ends of the price range. The first map present the resales occured from 2005-2020 with price greater than SGD 800,000. The second map shows the resales since 2005-2020 with price less than SGD 150,000. The maps will be rendered with red circles.

In [15]:
renderMap(data[(data['resale_year']>2005) & (data['resale_price'] > 800000)], 20, "#AA0000")
sgmap.save("resale-highprice-map.html")
sgmap

In [13]:
renderMap(data[(data['resale_year']>2005) & (data['resale_price'] < 150000)], 20, "#AA0000")
sgmap.save("resale-lowprice-map.html")
sgmap

The maps show that most of the more expensive HDB resale occurred around the central region of Singapore, while the less expensive ones usually lie towards the outskirts.

One plausible factor to the higher price is the proximity to key amenities and employment centers. The central region is home to Singapore's central business district, where many high-paying jobs are located. This concentration of employment opportunities attracts professionals and individuals with higher purchasing power, leading to a higher demand for housing in the area and subsequently a higher price. The perception of prestige and convenience often associated with central regions can influence the pricing.

On the other hand, the outskirts generally have fewer amenities and are farther away from major employment centers. This distance can lead to lower demand for housing in these areas compared to the central region. Additionally, the lower cost of land in the outskirts contributes to more affordable housing prices. Developers may find it more cost-effective to build in these areas, leading to a greater supply of housing and potentially lower prices.